# Análisis de Fragmentos MEG - Generación de DataFrame
## LibriBrain Competition - Conversión de Datos

Este notebook convierte fragmentos de señales MEG en un DataFrame de pandas con:
- **Filas**: Fragmentos temporales de la grabación
- **Columnas**: Potencia espectral por banda de frecuencia para cada canal seleccionado
- **Configurables**: Duración del fragmento, bandas de frecuencia, canales a analizar

**Frecuencia de muestreo**: 250 Hz

In [3]:
# Importaciones necesarias
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy import signal
import json
import os
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Configuración
SFREQ = 250  # Frecuencia de muestreo (Hz)

print("✅ Librerías importadas correctamente")
print(f"📊 Frecuencia de muestreo: {SFREQ} Hz")

✅ Librerías importadas correctamente
📊 Frecuencia de muestreo: 250 Hz


In [5]:
# Configuración de rutas
base_path = "c:/Users/jp/Repositorios/LibriBrian_Competition/libribrain"
hdf5_file_path = f"{base_path}/data/Sherlock1/derivatives/serialised/sub-0_ses-1_task-Sherlock1_run-1_proc-bads+headpos+sss+notch+bp+ds_meg.h5"
sensor_file_path = f"{base_path}/sensor_xyz.json"

# Verificar estructura de datos MEG SIN cargar en memoria
print("🔄 Verificando estructura de datos MEG...")
with h5py.File(hdf5_file_path, "r") as f:
    data_shape = f["data"].shape
    times_shape = f["times"].shape
    # Solo cargar los timestamps (son pequeños)
    times = f["times"][:]

print(f"📊 Estructura de datos (SIN cargar en memoria):")
print(f"   - Shape datos: {data_shape}")
print(f"   - Duración total: {times[-1]:.2f} segundos ({times[-1]/60:.2f} minutos)")
print(f"   - Número de sensores: {data_shape[0]}")
print(f"   - Número de muestras temporales: {data_shape[1]}")
print(f"   - Tamaño estimado en memoria: {(data_shape[0] * data_shape[1] * 8) / 1024**3:.2f} GB")

# Sensores importantes para detección de habla
SPEECH_SENSORS = [18, 20, 22, 23, 45, 120, 138, 140, 142, 143, 145,
                  146, 147, 149, 175, 176, 177, 179, 180, 198, 271, 272, 275]

print(f"🎯 Sensores de habla disponibles: {len(SPEECH_SENSORS)}")
print(f"   Índices: {SPEECH_SENSORS}")

# Variables globales para acceso eficiente
HDF5_FILE_PATH = hdf5_file_path
TOTAL_SAMPLES = data_shape[1]
TOTAL_CHANNELS = data_shape[0]

🔄 Verificando estructura de datos MEG...
📊 Estructura de datos (SIN cargar en memoria):
   - Shape datos: (306, 278000)
   - Duración total: 1112.00 segundos (18.53 minutos)
   - Número de sensores: 306
   - Número de muestras temporales: 278000
   - Tamaño estimado en memoria: 0.63 GB
🎯 Sensores de habla disponibles: 23
   Índices: [18, 20, 22, 23, 45, 120, 138, 140, 142, 143, 145, 146, 147, 149, 175, 176, 177, 179, 180, 198, 271, 272, 275]


In [6]:
def generate_meg_fragments_dataframe(
    hdf5_file_path: str,
    fragment_duration: float,
    selected_channels: List[int],
    frequency_bands: Dict[str, Tuple[float, float]],
    start_time: float = 0.0,
    end_time: Optional[float] = None,
    overlap_ratio: float = 0.0,
    sfreq: int = 250,
    max_fragments: Optional[int] = None
) -> pd.DataFrame:
    """
    Genera un DataFrame con fragmentos de señales MEG y sus potencias espectrales por banda.
    OPTIMIZADO: Carga solo los fragmentos necesarios, no todo el archivo en memoria.
    
    Parámetros:
    -----------
    hdf5_file_path : str
        Ruta al archivo HDF5 con los datos MEG
    fragment_duration : float
        Duración de cada fragmento en segundos
    selected_channels : List[int]
        Lista de índices de canales a analizar
    frequency_bands : Dict[str, Tuple[float, float]]
        Diccionario con bandas de frecuencia {'nombre': (freq_min, freq_max)}
    start_time : float, default=0.0
        Tiempo de inicio en segundos
    end_time : Optional[float], default=None
        Tiempo de fin en segundos (None = hasta el final)
    overlap_ratio : float, default=0.0
        Proporción de solapamiento entre fragmentos (0.0 = sin solapamiento, 0.5 = 50% solapamiento)
    sfreq : int, default=250
        Frecuencia de muestreo en Hz
    max_fragments : Optional[int], default=None
        Máximo número de fragmentos a procesar (None = todos)
    
    Returns:
    --------
    pd.DataFrame
        DataFrame con filas=fragmentos y columnas=potencias espectrales por banda y canal
    """
    
    # Abrir archivo HDF5 para obtener información básica
    with h5py.File(hdf5_file_path, "r") as f:
        total_samples = f["data"].shape[1]
        total_channels = f["data"].shape[0]
    
    # Convertir tiempos a muestras
    fragment_samples = int(fragment_duration * sfreq)
    start_sample = int(start_time * sfreq)
    end_sample = int(end_time * sfreq) if end_time is not None else total_samples
    
    # Calcular paso entre fragmentos considerando solapamiento
    step_samples = int(fragment_samples * (1 - overlap_ratio))
    
    # Validaciones
    if fragment_samples <= 0:
        raise ValueError("La duración del fragmento debe ser positiva")
    if start_sample >= end_sample:
        raise ValueError("El tiempo de inicio debe ser menor que el tiempo de fin")
    if end_sample > total_samples:
        raise ValueError("El tiempo de fin excede la duración de los datos")
    if max(selected_channels) >= total_channels:
        raise ValueError(f"Canal máximo ({max(selected_channels)}) excede el número de canales ({total_channels})")
    
    # Calcular fragmentos a procesar
    fragment_starts = list(range(start_sample, end_sample - fragment_samples + 1, step_samples))
    if max_fragments:
        fragment_starts = fragment_starts[:max_fragments]
    
    print(f"📊 Configuración del análisis EFICIENTE:")
    print(f"   - Duración del fragmento: {fragment_duration}s ({fragment_samples} muestras)")
    print(f"   - Canales seleccionados: {len(selected_channels)}")
    print(f"   - Bandas de frecuencia: {len(frequency_bands)}")
    print(f"   - Solapamiento: {overlap_ratio*100:.1f}%")
    print(f"   - Fragmentos a procesar: {len(fragment_starts)}")
    print(f"   - Rango temporal: {start_time:.1f}s - {end_time or end_sample/sfreq:.1f}s")
    print(f"   - Memoria por fragmento: ~{(len(selected_channels) * fragment_samples * 8) / 1024**2:.2f} MB")
    
    # Procesar fragmentos uno por uno
    fragments_data = []
    
    with h5py.File(hdf5_file_path, "r") as f:
        for fragment_idx, start_idx in enumerate(fragment_starts):
            fragment_start_time = start_idx / sfreq
            fragment_end_time = (start_idx + fragment_samples) / sfreq
            
            # CARGAR SOLO EL FRAGMENTO ACTUAL - ¡SIN mantener todo en memoria!
            fragment = f["data"][selected_channels, start_idx:start_idx + fragment_samples]
            
            # Calcular FFT para cada canal
            fragment_powers = {}
            
            for i, channel_idx in enumerate(selected_channels):
                channel_signal = fragment[i, :]
                
                # FFT
                fft_values = np.abs(fft(channel_signal))
                frequencies = fftfreq(len(channel_signal), 1/sfreq)
                
                # Tomar solo frecuencias positivas
                positive_mask = frequencies >= 0
                fft_positive = fft_values[positive_mask]
                freq_positive = frequencies[positive_mask]
                
                # Calcular potencia por banda
                for band_name, (low_freq, high_freq) in frequency_bands.items():
                    freq_mask = (freq_positive >= low_freq) & (freq_positive <= high_freq)
                    
                    if np.any(freq_mask):
                        # Potencia espectral promedio en la banda
                        power = np.mean(fft_positive[freq_mask]**2)
                    else:
                        power = 0.0
                    
                    # Crear nombre de columna
                    col_name = f"Ch{channel_idx}_{band_name}"
                    fragment_powers[col_name] = power
            
            # Agregar información temporal
            fragment_powers['fragment_start_time'] = fragment_start_time
            fragment_powers['fragment_end_time'] = fragment_end_time
            fragment_powers['fragment_duration'] = fragment_duration
            fragment_powers['fragment_idx'] = fragment_idx
            
            fragments_data.append(fragment_powers)
            
            # Progreso cada 100 fragmentos
            if (fragment_idx + 1) % 100 == 0:
                print(f"   Procesados {fragment_idx + 1}/{len(fragment_starts)} fragmentos...")
    
    # Crear DataFrame
    df = pd.DataFrame(fragments_data)
    
    # Reorganizar columnas: información temporal primero, luego potencias por canal y banda
    temporal_cols = ['fragment_idx', 'fragment_start_time', 'fragment_end_time', 'fragment_duration']
    power_cols = [col for col in df.columns if col not in temporal_cols]
    df = df[temporal_cols + sorted(power_cols)]
    
    print(f"✅ DataFrame generado EFICIENTEMENTE:")
    print(f"   - Fragmentos: {len(df)}")
    print(f"   - Columnas: {len(df.columns)}")
    print(f"   - Columnas de potencia: {len(power_cols)}")
    print(f"   - Memoria del DataFrame: ~{df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    return df

print("🔧 Función generate_meg_fragments_dataframe definida")

🔧 Función generate_meg_fragments_dataframe definida


In [7]:
# Configuraciones predefinidas de bandas de frecuencia

# Bandas neurales clásicas
NEURAL_BANDS = {
    'Delta': (0.5, 4.0),
    'Theta': (4.0, 8.0),
    'Alpha': (8.0, 13.0),
    'Beta': (13.0, 30.0),
    'Gamma': (30.0, 50.0)
}

# Bandas ultra-cortas (basadas en el análisis anterior)
ULTRA_SHORT_BANDS = {
    'Band_0-10Hz': (0, 10),
    'Band_10-20Hz': (10, 20), 
    'Band_20-30Hz': (20, 30),
    'Band_30-40Hz': (30, 40),
    'Band_40-50Hz': (40, 50),
    'Band_50-60Hz': (50, 60),
    'Band_60-70Hz': (60, 70),
    'Band_70-80Hz': (70, 80),
    'Band_80-90Hz': (80, 90),
    'Band_90-100Hz': (90, 100)
}

# Bandas personalizadas para habla
SPEECH_BANDS = {
    'LowFreq': (1.0, 10.0),
    'SpeechLow': (10.0, 25.0),
    'SpeechMid': (25.0, 50.0),
    'SpeechHigh': (50.0, 80.0),
    'HighGamma': (80.0, 120.0)
}

print("📊 Bandas de frecuencia predefinidas:")
print(f"   - Bandas neurales: {list(NEURAL_BANDS.keys())}")
print(f"   - Bandas ultra-cortas: {len(ULTRA_SHORT_BANDS)} bandas de 10Hz")
print(f"   - Bandas de habla: {list(SPEECH_BANDS.keys())}")

📊 Bandas de frecuencia predefinidas:
   - Bandas neurales: ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']
   - Bandas ultra-cortas: 10 bandas de 10Hz
   - Bandas de habla: ['LowFreq', 'SpeechLow', 'SpeechMid', 'SpeechHigh', 'HighGamma']


In [10]:
# EJEMPLO DE USO: Generación de DataFrame con fragmentos configurables
# ================================================================

# CONFIGURACIÓN DEL USUARIO (modifica estos parámetros según tus necesidades)
# -------------------------------------------------------------------------

# 1. Duración del fragmento (en segundos)
fragment_duration = 2.0  # Cambia aquí: 0.5, 1.0, 2.0, 5.0, etc.

# 2. Selección de canales (puedes usar nombres o índices)
# Opción 1: Por nombres de sensores (si tienes los nombres)
# selected_channels = ['MEG0113', 'MEG0122', 'MEG0132']  
# Opción 2: Por índices (más directo)
selected_channels = [0, 1, 2, 10, 20, 30]  # Primeros canales como ejemplo

# 3. Bandas de frecuencia a analizar
# Puedes elegir uno de los diccionarios predefinidos o crear el tuyo
frequency_bands = NEURAL_BANDS  # Cambia por: ULTRA_SHORT_BANDS, SPEECH_BANDS, o tu propio diccionario

# 4. Otros parámetros opcionales
overlap_ratio = 0.0  # Sin solapamiento (0.0), 50% solapamiento (0.5), etc.
include_temporal_info = True  # Incluir información temporal en el DataFrame

print("Configuración seleccionada:")
print(f"- Duración del fragmento: {fragment_duration} segundos")
print(f"- Número de canales seleccionados: {len(selected_channels)}")
print(f"- Canales: {selected_channels}")
print(f"- Bandas de frecuencia: {list(frequency_bands.keys())}")
print(f"- Solapamiento: {overlap_ratio*100:.0f}%")

# GENERACIÓN DEL DATAFRAME
# ------------------------
print("\nGenerando DataFrame de fragmentos...")

# Llamada a la función principal OPTIMIZADA
fragments_df = generate_meg_fragments_dataframe(
    hdf5_file_path=HDF5_FILE_PATH,  # Pasamos la ruta, no los datos
    fragment_duration=fragment_duration,
    selected_channels=selected_channels,
    frequency_bands=frequency_bands,
    sfreq=SFREQ,
    overlap_ratio=overlap_ratio,
    max_fragments=500  # Limitar para pruebas (quitar este parámetro para procesar todos)
)

# INFORMACIÓN DEL RESULTADO
# -------------------------
print(f"\n✅ DataFrame generado exitosamente!")
print(f"📊 Forma del DataFrame: {fragments_df.shape}")
print(f"📝 Columnas: {len(fragments_df.columns)}")
print(f"⏱️  Número de fragmentos: {len(fragments_df)}")

# Mostrar las primeras columnas para verificar la estructura
print(f"\nPrimeras 5 columnas del DataFrame:")
print(fragments_df.columns[:5].tolist())

# Mostrar una muestra del DataFrame
print(f"\nPrimeras 3 filas del DataFrame:")
print(fragments_df.head(3))

# Información sobre las columnas de potencia
power_columns = [col for col in fragments_df.columns if col.startswith('Ch')]
print(f"\nColumnas de potencia generadas: {len(power_columns)}")
print(f"Ejemplo de columnas de potencia: {power_columns[:3]}...")

# ESTADÍSTICAS BÁSICAS
# --------------------
if len(power_columns) >= 3:
    print(f"\nEstadísticas básicas de las primeras columnas de potencia:")
    power_sample = fragments_df[power_columns[:3]]
    print(power_sample.describe())
else:
    print(f"\n⚠️ Solo se encontraron {len(power_columns)} columnas de potencia.")
    if len(power_columns) > 0:
        print("Estadísticas básicas de las columnas disponibles:")
        power_sample = fragments_df[power_columns]
        print(power_sample.describe())

print(f"\n🎯 El DataFrame está listo para análisis posteriores!")
print(f"   Puedes usar diferentes algoritmos de ML sobre este DataFrame.")
print(f"   Cada fila representa un fragmento de {fragment_duration}s con sus características espectrales.")

Configuración seleccionada:
- Duración del fragmento: 2.0 segundos
- Número de canales seleccionados: 6
- Canales: [0, 1, 2, 10, 20, 30]
- Bandas de frecuencia: ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']
- Solapamiento: 0%

Generando DataFrame de fragmentos...
📊 Configuración del análisis EFICIENTE:
   - Duración del fragmento: 2.0s (500 muestras)
   - Canales seleccionados: 6
   - Bandas de frecuencia: 5
   - Solapamiento: 0.0%
   - Fragmentos a procesar: 500
   - Rango temporal: 0.0s - 1112.0s
   - Memoria por fragmento: ~0.02 MB
   Procesados 100/500 fragmentos...
   Procesados 200/500 fragmentos...
   Procesados 300/500 fragmentos...
   Procesados 400/500 fragmentos...
   Procesados 500/500 fragmentos...
✅ DataFrame generado EFICIENTEMENTE:
   - Fragmentos: 500
   - Columnas: 34
   - Columnas de potencia: 30
   - Memoria del DataFrame: ~0.07 MB

✅ DataFrame generado exitosamente!
📊 Forma del DataFrame: (500, 34)
📝 Columnas: 34
⏱️  Número de fragmentos: 500

Primeras 5 columnas de

In [12]:
# 📊 VISUALIZACIÓN COMPLETA DEL DATAFRAME
# =========================================

print("🔍 ANÁLISIS DETALLADO DEL DATAFRAME GENERADO")
print("="*60)

# 1. INFORMACIÓN GENERAL
print(f"\n📋 INFORMACIÓN GENERAL:")
print(f"   - Forma del DataFrame: {fragments_df.shape}")
print(f"   - Tipos de datos:")
print(fragments_df.dtypes)

print(f"\n📊 MEMORIA Y RENDIMIENTO:")
memory_usage = fragments_df.memory_usage(deep=True)
total_memory = memory_usage.sum() / 1024**2
print(f"   - Memoria total: {total_memory:.2f} MB")
print(f"   - Memoria por columna (MB):")
for col in fragments_df.columns[:10]:  # Mostrar primeras 10
    col_memory = memory_usage[col] / 1024**2
    print(f"     {col}: {col_memory:.3f} MB")
if len(fragments_df.columns) > 10:
    print(f"     ... y {len(fragments_df.columns)-10} columnas más")

# 2. ESTRUCTURA DE COLUMNAS
print(f"\n🗂️ ESTRUCTURA DE COLUMNAS:")
temporal_cols = ['fragment_idx', 'fragment_start_time', 'fragment_end_time', 'fragment_duration']
power_cols = [col for col in fragments_df.columns if col.startswith('Ch')]

print(f"   - Columnas temporales ({len(temporal_cols)}): {temporal_cols}")
print(f"   - Columnas de potencia ({len(power_cols)}): {power_cols[:5]}..." if len(power_cols) > 5 else f"   - Columnas de potencia ({len(power_cols)}): {power_cols}")

# 3. MUESTRA DEL DATAFRAME
print(f"\n📋 PRIMERAS 5 FILAS DEL DATAFRAME:")
print(fragments_df.head())

print(f"\n📋 ÚLTIMAS 5 FILAS DEL DATAFRAME:")
print(fragments_df.tail())

# 4. ESTADÍSTICAS DESCRIPTIVAS
print(f"\n📈 ESTADÍSTICAS DESCRIPTIVAS:")
if len(power_cols) > 0:
    print("Columnas temporales:")
    print(fragments_df[temporal_cols].describe())
    
    print(f"\nPrimeras 5 columnas de potencia:")
    sample_power_cols = power_cols[:5]
    print(fragments_df[sample_power_cols].describe())
else:
    print("⚠️ No se encontraron columnas de potencia")

# 5. RANGO DE VALORES
print(f"\n📊 RANGOS DE VALORES:")
print(f"   - Tiempo de inicio: {fragments_df['fragment_start_time'].min():.2f}s - {fragments_df['fragment_start_time'].max():.2f}s")
print(f"   - Tiempo de fin: {fragments_df['fragment_end_time'].min():.2f}s - {fragments_df['fragment_end_time'].max():.2f}s")

if len(power_cols) > 0:
    power_values = fragments_df[power_cols].values.flatten()
    print(f"   - Potencia mínima: {power_values.min():.2e}")
    print(f"   - Potencia máxima: {power_values.max():.2e}")
    print(f"   - Potencia promedio: {power_values.mean():.2e}")

# 6. INFORMACIÓN POR CANAL Y BANDA
print(f"\n🎵 ANÁLISIS POR CANAL Y BANDA:")
if len(power_cols) > 0:
    # Extraer información de canales y bandas
    channels = set()
    bands = set()
    for col in power_cols:
        if col.startswith('Ch'):
            parts = col.split('_')
            if len(parts) >= 2:
                channels.add(parts[0])
                bands.add('_'.join(parts[1:]))
    
    print(f"   - Canales únicos: {len(channels)} ({sorted(list(channels))[:5]}...)" if len(channels) > 5 else f"   - Canales únicos: {len(channels)} ({sorted(list(channels))})")
    print(f"   - Bandas únicas: {len(bands)} ({sorted(list(bands))})")

# 7. VERIFICACIÓN DE CALIDAD
print(f"\n✅ VERIFICACIÓN DE CALIDAD:")
print(f"   - Valores nulos: {fragments_df.isnull().sum().sum()}")
print(f"   - Valores infinitos: {np.isinf(fragments_df.select_dtypes(include=[np.number])).sum().sum()}")
print(f"   - Filas duplicadas: {fragments_df.duplicated().sum()}")

# 8. RESUMEN FINAL
print(f"\n🎯 RESUMEN FINAL:")
print(f"   ✅ DataFrame con {fragments_df.shape[0]} fragmentos de {fragment_duration}s cada uno")
print(f"   ✅ {len(power_cols)} características espectrales (potencia por banda/canal)")
print(f"   ✅ Rango temporal: {fragments_df['fragment_start_time'].min():.1f}s - {fragments_df['fragment_end_time'].max():.1f}s")
print(f"   ✅ Memoria eficiente: {total_memory:.2f} MB")
print(f"   ✅ Datos listos para análisis y machine learning")

print(f"\n💡 PRÓXIMOS PASOS SUGERIDOS:")
print(f"   - Guardar DataFrame: fragments_df.to_csv('meg_fragments.csv')")
print(f"   - Análisis estadístico: correlaciones, distribuciones")
print(f"   - Machine Learning: clasificación, clustering, detección de anomalías")
print(f"   - Visualizaciones: heatmaps, series temporales, distribuciones")

🔍 ANÁLISIS DETALLADO DEL DATAFRAME GENERADO

📋 INFORMACIÓN GENERAL:
   - Forma del DataFrame: (500, 34)
   - Tipos de datos:
fragment_idx             int64
fragment_start_time    float64
fragment_end_time      float64
fragment_duration      float64
Ch0_Alpha              float32
Ch0_Beta               float32
Ch0_Delta              float32
Ch0_Gamma              float32
Ch0_Theta              float32
Ch10_Alpha             float32
Ch10_Beta              float32
Ch10_Delta             float32
Ch10_Gamma             float32
Ch10_Theta             float32
Ch1_Alpha              float32
Ch1_Beta               float32
Ch1_Delta              float32
Ch1_Gamma              float32
Ch1_Theta              float32
Ch20_Alpha             float32
Ch20_Beta              float32
Ch20_Delta             float32
Ch20_Gamma             float32
Ch20_Theta             float32
Ch2_Alpha              float32
Ch2_Beta               float32
Ch2_Delta              float32
Ch2_Gamma              float32
Ch2_The

In [13]:
# 🖨️ IMPRIMIR/MOSTRAR EL DATAFRAME
# ==================================

print("🖨️ MOSTRANDO EL DATAFRAME DE FRAGMENTOS MEG")
print("="*50)

# OPCIÓN 1: Mostrar todo el DataFrame (para DataFrames pequeños)
print("\n1️⃣ DATAFRAME COMPLETO:")
print("Nota: Si el DataFrame es muy grande, solo se mostrarán las primeras y últimas filas")
display(fragments_df)

# OPCIÓN 2: Mostrar primeras 20 filas
print(f"\n2️⃣ PRIMERAS 20 FILAS:")
print(fragments_df.head(20))

# OPCIÓN 3: Mostrar solo columnas específicas (temporal + algunas de potencia)
print(f"\n3️⃣ VISTA COMPACTA (columnas temporales + 3 columnas de potencia):")
temporal_cols = ['fragment_idx', 'fragment_start_time', 'fragment_end_time', 'fragment_duration']
power_cols = [col for col in fragments_df.columns if col.startswith('Ch')]
selected_cols = temporal_cols + power_cols[:3]
print(fragments_df[selected_cols].head(10))

# OPCIÓN 4: Información de todas las columnas
print(f"\n4️⃣ TODAS LAS COLUMNAS DEL DATAFRAME:")
print("Columnas disponibles:")
for i, col in enumerate(fragments_df.columns):
    print(f"   {i+1:2d}. {col}")

# OPCIÓN 5: Mostrar rango específico de filas
print(f"\n5️⃣ FILAS 10-20 (ejemplo de rango específico):")
if len(fragments_df) >= 20:
    print(fragments_df.iloc[10:20])
else:
    print("El DataFrame tiene menos de 20 filas")

# OPCIÓN 6: Resumen en formato string para copiar/pegar
print(f"\n6️⃣ RESUMEN PARA COPIAR:")
print(f"DataFrame shape: {fragments_df.shape}")
print(f"Columnas: {list(fragments_df.columns)}")
print("\nPrimeras 5 filas como string:")
print(fragments_df.head().to_string())

print(f"\n💡 OPCIONES ADICIONALES PARA GUARDAR:")
print(f"   - fragments_df.to_csv('meg_fragments.csv', index=False)  # Guardar como CSV")
print(f"   - fragments_df.to_excel('meg_fragments.xlsx', index=False)  # Guardar como Excel")
print(f"   - fragments_df.to_parquet('meg_fragments.parquet')  # Guardar como Parquet (eficiente)")
print(f"   - print(fragments_df.to_string())  # Imprimir todo como texto")

🖨️ MOSTRANDO EL DATAFRAME DE FRAGMENTOS MEG

1️⃣ DATAFRAME COMPLETO:
Nota: Si el DataFrame es muy grande, solo se mostrarán las primeras y últimas filas


,fragment_idx,fragment_start_time,fragment_end_time,fragment_duration,Ch0_Alpha,Ch0_Beta,Ch0_Delta,Ch0_Gamma,Ch0_Theta,Ch10_Alpha,...,Ch2_Alpha,Ch2_Beta,Ch2_Delta,Ch2_Gamma,Ch2_Theta,Ch30_Alpha,Ch30_Beta,Ch30_Delta,Ch30_Gamma,Ch30_Theta
0,0,0.0,2.0,2.0,9.892111e-23,4.730418e-23,2.626055e-21,1.422835e-23,2.081702e-22,1.157648e-20,...,1.086404e-20,1.201781e-20,8.632333e-19,4.563371e-21,1.096019e-19,6.715202e-23,1.861358e-23,8.442121e-22,6.763675e-24,1.259162e-22
1,1,2.0,4.0,2.0,4.488864e-23,1.839891e-23,2.260324e-21,1.034728e-23,9.125779e-23,2.024121e-20,...,1.985997e-20,7.169200e-21,9.983247e-20,6.137801e-21,2.481629e-20,6.714506e-23,2.188753e-23,4.643470e-22,8.198354e-24,5.187816e-23
2,2,4.0,6.0,2.0,6.425056e-23,2.477917e-23,1.091769e-21,8.224824e-24,7.629690e-23,1.521651e-20,...,1.558484e-20,5.292804e-21,1.149036e-19,2.867859e-21,1.146615e-20,3.011551e-23,1.512733e-23,2.316521e-22,4.705713e-24,6.257002e-23
3,3,6.0,8.0,2.0,6.945857e-23,3.535342e-23,6.379245e-22,9.401255e-24,1.348052e-22,2.004856e-20,...,2.902653e-20,9.458900e-21,1.891201e-19,3.601698e-21,2.340977e-20,1.301416e-22,4.681746e-23,4.448007e-22,7.121774e-24,2.249855e-23
4,4,8.0,10.0,2.0,5.075640e-23,2.215059e-23,1.338289e-21,8.016750e-24,1.367159e-22,2.519302e-20,...,1.365329e-20,6.129530e-21,8.964151e-20,2.843236e-21,2.001717e-20,6.517296e-23,2.399363e-23,7.700629e-22,4.211719e-24,6.264476e-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,990.0,992.0,2.0,7.557645e-23,3.520449e-23,8.842497e-23,7.892418e-24,1.036707e-22,2.378668e-20,...,4.231042e-20,1.201309e-20,7.337407e-20,2.721564e-21,3.401780e-20,2.553982e-22,6.788940e-23,3.300020e-22,9.223118e-24,2.273224e-22
496,496,992.0,994.0,2.0,7.993079e-23,3.108552e-23,9.281770e-22,8.249201e-24,2.236741e-22,1.426099e-20,...,6.485993e-20,1.375771e-20,2.436486e-19,3.784170e-21,5.170139e-20,2.294641e-22,4.549867e-23,3.223634e-22,6.051770e-24,2.194125e-22
497,497,994.0,996.0,2.0,6.262688e-23,2.744708e-23,4.151701e-22,8.829745e-24,6.543387e-23,1.174478e-20,...,3.876177e-20,9.574633e-21,2.921095e-20,3.670834e-21,3.721544e-20,7.726403e-23,3.246801e-23,6.210711e-23,6.626777e-24,5.702261e-23
498,498,996.0,998.0,2.0,7.280142e-23,3.523589e-23,9.542265e-22,9.948237e-24,2.146342e-22,2.409448e-20,...,1.968799e-20,1.015782e-20,2.132318e-19,4.987147e-21,4.307832e-20,8.042350e-23,3.363395e-23,2.636458e-22,6.589716e-24,1.016880e-22



2️⃣ PRIMERAS 20 FILAS:
    fragment_idx  fragment_start_time  fragment_end_time  fragment_duration  \
0              0                  0.0                2.0                2.0   
1              1                  2.0                4.0                2.0   
2              2                  4.0                6.0                2.0   
3              3                  6.0                8.0                2.0   
4              4                  8.0               10.0                2.0   
5              5                 10.0               12.0                2.0   
6              6                 12.0               14.0                2.0   
7              7                 14.0               16.0                2.0   
8              8                 16.0               18.0                2.0   
9              9                 18.0               20.0                2.0   
10            10                 20.0               22.0                2.0   
11            11            